# Introduction to [`pymoo`](https://pymoo.org/index.html)

In [ ]:
# Copyright (c) 2023 Chen Xueqiang
#
# -*- coding:utf-8 -*-
# @Script: intro_pymoo.ipynb
# @Author: Chen Xueqiang
# @Email: carlchenxq@qq.com
# @Create At: 2023-05-30 16:28:43
# @Last Modified By: Chen Xueqiang
# @Last Modified At: 2023-05-30 16:28:43
# @Description: A brieff introduction to `pymoo`.


credit: https://www.kaggle.com/code/shikachen/an-intro-to-pymoo-solving-optimization-problems/notebook
In this notebook, we are going to explore how to use `pymoo` to solve optimization problems (single & multiple). Like Sklearn to machine learning, PyTorch & TF for deep learning. `pymoo` is an opensource multi-objective optimization framework which simplifies the implementation of several MO-Optimization Algorithms. Including NSGA-II, MOEA/D and so on. 

We are going to solve 2 different benchmark problems to show how to use `pymoo`.

Let's move on! 

## Custom Single Object Problem 


### McCormick function - non-constrained

- Formula: $f(x,y)=\sin \left(x+y\right)+\left(x-y\right)^{2}-1.5x+2.5y+1$

- Global Minimun: $f(-0.54719,-1.54719)=-1.9133$

- Search Domain: $-1.5\leq x\leq 4, -3\leq y\leq 4$

#### Problem Defination (code)

- To define a problem, the most recommended way is create a Python class, a thoughout tutorial can be found in the [official site](https://pymoo.org/problems/definition.html). 

- Define the problem using 2 diffetent Problem-Type (of pymoo lib). 

    The regular Problem-Type handles the solutions all togather in `_evaluate` function. 

    For this example: We have a problem with 2 different variables `x` and `y` and suppose we have a population of 100. 

    The regular Problem-Type would pass a `x` with the shape (100, 2) to the `_evaluate` function. 

    The ElementWise Problem-Type, would call the `_evaluate` 100 times and pass a (2,) solution to the `_evaluate` each time. 

- Coding Rule(s)

    In `_evaluate`, we are going to fill a dictionary, which contains Function Result, Inequal Constrains and Equal Constrains

    in `out["F"]`, `out["G"]` and `out["H"]`: 

    |Dict|Meaning|
    |---|---|
    |`out["F"]`|Function Results|
    |`out["G"]`|Inequal Constrains|
    |`out["H"]`|Equal Constrains|

    each of them is a list of values (without list if there are only 1 for each of them). 

Let's go with some exampls. 

In [ ]:
# install pymoo
! pip install -U pymoo

In [ ]:
# import some libs
import numpy as np
from pymoo.core.problem import Problem
from pymoo.core.problem import ElementwiseProblem
from pymoo.algorithms.soo.nonconvex.pso import PSO
from pymoo.optimize import minimize

In [ ]:

class McCormick(Problem):
    """Regular type of Problem defination for McCormic Function. 

    Args:

    functions:
        _evaluate: Get the values of target function and fill the values into a dict called `out`. 
        cal_cormick: To compute the value of target function. 
    """
    def __init__(self):
        # basic setups
        # n_var -> number of variables
        # n_obj -> number of objectives
        # xl -> lower boundaries of variables
        # xu -> upper boundaries of variables
        super().__init__(n_var=2, n_obj=1, xl=np.array([-1.5, -3]), xu=np.array([4, 4]))

    def _evaluate(self, x, out, *args, **kwargs):
        # x, out are needed, it's basic
        # x -> solutions, for this kind of Problem, the x is a (population, n_var) matrix. 
        # out -> the corresponding results, including function values, constrain values and so on. 
        #        we only fill what we need. 
        out["F"] = self.cal_cormick(x)  # fill the function values of each solution

    def cal_cormick(self, x):
        """compute the McCormic Function

        Args:
            x: the solutions (all population)

        Returns:
            f: the value of McCormic function of solutions
        """
        f = np.sin(x[:, 0] + x[:, 1]) + (x[:, 0] - x[:, 1]) ** 2 - 1.5 * x[:, 0] + 2.5 * x[:, 1] + 1
        return f
    
class McCormickEle(ElementwiseProblem):
    def __init__(self, **kwargs):
        super().__init__(n_var=2, n_obj=1, xl=np.array([-1.5, -3]), xu=np.array([4, 4]), **kwargs)

    def _evaluate(self, x, out, *args, **kwargs):
        # in ElementwiseProblem, the x is a single solution
        out["F"] = self.cal_cormick(x)

    def cal_cormick(self, x):
        f = np.sin(x[0] + x[1]) + (x[0] - x[1]) ** 2 - 1.5 * x[ 0] + 2.5 * x[1] + 1
        return f

In this problem class, we define the `_evaluate` to fill the dictionary `out["F"]`. 

For calculating the target function, we define another function `cal_cormick` to do the computation and return the value of the target function. 

Let's find the optimal solution for this problem using `pymoo` Algorithms then. 

#### Fit using PSO (Regular)

In [ ]:
mc_cor = McCormick()  # assign a object of the problem we defined above

pso = PSO()  # assign a PSO with default hyper params

pso_res = minimize(  # fit the PSO
    mc_cor, 
    pso, 
    seed=1,
    verbose=True
    )

print("Best solution: \nX = %s\nF = %s" % (pso_res.X, pso_res.F))

*****

#### Fit using PSO (ElementWise)

In [ ]:
mc_cor_ele = McCormickEle()

pso_ele_res = minimize(
    mc_cor_ele, 
    pso, 
    seed=1, 
    verbose=True
    )

print("Best solution: \nX = %s\nF = %s" % (pso_ele_res.X, pso_ele_res.F))

*****

#### Fit using PatterSearch

In [ ]:
from pymoo.algorithms.soo.nonconvex.pattern import PatternSearch

ps = PatternSearch()

res = minimize(
    mc_cor,
    ps,
    verbose=True,
    seed=1
    )

print("Best solution found: \nX = %s\nF = %s" % (res.X, res.F))

*****

#### Parrallelization for ElementWise Problems

In [ ]:
from multiprocessing.pool import ThreadPool
from pymoo.core.problem import StarmapParallelization

# initialize the thread pool and create the runner
n_threads = 4
pool = ThreadPool(n_threads)
runner = StarmapParallelization(pool.starmap)

paral_mc_cor_ele = McCormickEle(elementwise_runner=runner)

res_paral = minimize(
    paral_mc_cor_ele, 
    pso, 
    seed=1, 
    verbose=False
    )

print("Best solution found: \nX = %s\nF = %s" % (res_paral.X, res_paral.F))

## Custom Multiple Object Problem with Scalar Value

In this part, we are going to explore how to define a problem with constrains. And, how to handle multiple objects. Where other setups are as same as the non-constrained problems. 

- Basic Knowledge

    **For all the constrains (equals and inequels)**: The baseline is `0`. 

    **For inequal constrains**: Always make it lower equal to `0`. 

    ex. If a constrain is $x + y \leq 1$, we handle it as $x + y - 1 \leq 0$

- Coding Rules

    inequal: `out["G"] = [g_1, g_2,..., g_n]`

    equal:   `out["H"] = [h_1, h_2,..., h_n]`

### ZDT3

Defination: 

${\displaystyle {\text{Minimize}}={\begin{cases}f_{1}\left({\boldsymbol {x}}\right)=x_{1}\\f_{2}\left({\boldsymbol {x}}\right)=g\left({\boldsymbol {x}}\right)h\left(f_{1}\left({\boldsymbol {x}}\right),g\left({\boldsymbol {x}}\right)\right)\\g\left({\boldsymbol {x}}\right)=1+{\frac {9}{n-1}}\sum _{i=2}^{n}x_{i}\\h\left(f_{1}\left({\boldsymbol {x}}\right),g\left({\boldsymbol {x}}\right)\right)=1-{\sqrt {\frac {f_{1}\left({\boldsymbol {x}}\right)}{g\left({\boldsymbol {x}}\right)}}}-\left({\frac {f_{1}\left({\boldsymbol {x}}\right)}{g\left({\boldsymbol {x}}\right)}}\right)\sin \left(10\pi f_{1}\left({\boldsymbol {x}}\right)\right)\end{cases}}}, 0\leq x_{i}\leq 1, 1\leq n\leq 30.$

In [ ]:
class ZDT3(Problem):
    def __init__(self, n_var):
        assert n_var >= 1 and n_var <= 30, "n_var should be in [1,30]"
        
        super().__init__(n_var=n_var,
                         n_obj=2,
                         xl=0,
                         xu=1)

        self._n_var = n_var

    def _evaluate(self, x, out, *args, **kwargs):
        f1 = x[:, 0]
        g = 1 + (9 / (self._n_var - 1)) * np.sum(x[:, 1:])
        h = 1 - np.sqrt(f1 / g) - (f1 / g) * np.sin(10 * np.pi * f1)
        f2 = g * h

        out["F"] = [f1, f2]

In [ ]:
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.sampling.rnd import FloatRandomSampling
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM

zdt3 = ZDT3(30)

nsga2 = NSGA2(
    pop_size=50,
    n_offsprings=10,
    sampling=FloatRandomSampling(),
    crossover=SBX(prob=0.9, eta=15),
    mutation=PM(eta=20),
    eliminate_duplicates=True
)

In [ ]:
zdt3_res = minimize(
    zdt3,
    nsga2,
    # ("n_gen", 20),
    seed=1,
    save_history=True,
    verbose=False
    )

X = zdt3_res.opt.get("X")
F = zdt3_res.opt.get("F")

In [ ]:
import matplotlib.pyplot as plt
xl, xu = zdt3.bounds()
plt.figure(figsize=(7, 5))
plt.scatter(X[:, 0], X[:, 1], s=30, facecolors='none', edgecolors='r')
plt.xlim(xl[0], xu[0])
plt.ylim(xl[1], xu[1])
plt.title("Design Space")
plt.show()

In [ ]:
plt.figure(figsize=(7, 5))
plt.scatter(F[:, 0], F[:, 1], s=30, facecolors='none', edgecolors='blue')
plt.title("Objective Space")
plt.show()

## The End

Based on the powerful tools in `pymoo`, we can solve a lot of optimization problems, especially multi-objective problems. Like hyper-parameters tuning, feature selection and any other practical or theoritical problems. 